<a href="https://colab.research.google.com/github/Penitto/risk_project1/blob/master/Iriski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Выделенные риск-факторы

1. Фондовый риск — риск снижения цены акций; __индекс РТС и МОЕКС__
2. Процентный риск — риск изменения процентных ставок; __процентные ставки__
3. Валютный риск — риск изменения курсов валют; __курс доллара и курс юаня__
4. Товарный риск — риск изменения цен товаров; __цены на нефть__


# Стохастическая модель динамики 

In [262]:
import numpy as np
import pandas as pd
import os
from scipy.interpolate import CubicSpline

In [263]:
class Portfolio():
    def __init__(self, bonds, shares, cur, risk):
        self.bonds = bonds
        self.shares = shares
        self.cur = cur
        self.risk = risk
        self.price = 0
        
    # Посчитать цену портфеля в определённый день
    def countPriceInDate(self, date):
        res = 0
        
        for i in self.bonds:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        for i in self.shares:
            res += i[3] * i[1]['<CLOSE>'].loc[date]
            
        
    # Посчитать объём портфеля в уе
    def countInitialValue(self):
        for i in range(len(self.bonds)):
            amount = self.bonds[i][2] / self.bonds[i][1]['<CLOSE>'][0]
            self.bonds[i] = (*self.bonds[i], amount)
            
        for i in range(len(self.shares)):
            amount = self.shares[i][2] / self.shares[i][1]['<CLOSE>'][0]
            self.shares[i] = (*self.shares[i], amount)
            
        for i in range(len(self.cur)):
            amount = self.cur[i][2] / self.cur[i][1]['<CLOSE>'][0]
            self.cur[i] = (*self.cur[i], amount)

In [299]:
# Загрузка данных в класс
shares = ['./shares/AFLT_160101_200101.csv', 
          './shares/GAZP_160101_200101.csv',
          './shares/GMKN_160101_200101.csv', 
          './shares/KMAZ_160101_200101.csv', 
          './shares/LKOH_160101_200101.csv', 
          './shares/PIKK_160101_200101.csv', 
          './shares/MGNT_160101_200101.csv', 
          './shares/RBCM_160101_200101.csv', 
          './shares/ROSN_160101_200101.csv', 
          './shares/SBER_160101_200101.csv']

shares_name = [i[9:13] for i in shares]

# Нужно интерполировать
bonds = ['./bonds/SU26212RMFS9_160101_200101.csv',
         './bonds/SU26205RMFS3_160101_200101.csv',
         './bonds/SU26207RMFS9_160101_200101.csv', 
         './bonds/SU26209RMFS5_160101_200101.csv', 
         './bonds/SU26211RMFS1_160101_200101.csv']

bonds_name = [i[8:20] for i in bonds]

currencies = ['./index/USD_RUB.csv', './index/CNY_RUB.csv']

currencies_name = [i[8:15] for i in currencies]

indexes = ['./index/ICE.BRN_160101_200101.csv', 
           './index/IMOEX_160101_200101.csv', 
           './index/RTSI_160101_200101.csv']

indexes_name = ['Brent', 'MOEX', 'RTSI']

zero_bond = './zerobond.csv'

days = 1010

In [300]:
risk_df = pd.read_csv(shares[0], index_col='<DATE>').drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1)
risk_df.index = pd.to_datetime(risk_df.index)
k = 1
for i in shares[1:]:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>': shares_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in bonds:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>' : bonds_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in currencies:
    tmp = pd.read_csv(i, index_col='Date') \
            .drop(['Open', 'High', 'Low', 'Change %'], axis=1) \
            .rename(columns={'Price' : currencies_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1
    
k = 0
for i in indexes:
    tmp = pd.read_csv(i, index_col='<DATE>') \
            .drop(['<TICKER>', '<PER>', '<TIME>', '<HIGH>', '<LOW>', '<VOL>', '<OPEN>'], axis=1) \
            .rename(columns={'<CLOSE>' : indexes_name[k]})
    tmp.index = pd.to_datetime(tmp.index)
    risk_df = risk_df.join(tmp, how='left')
    k += 1

risk_df = risk_df.rename(columns={'<CLOSE>' : shares_name[0]})

zero_bond_df = pd.read_csv(zero_bond, sep=';', index_col='Date')
zero_bond_df.index = pd.to_datetime(zero_bond_df.index)

risk_df = risk_df.join(zero_bond_df, how='left')
risk_df = risk_df.fillna(risk_df.mean(axis=0))

In [308]:
risk_df

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,...,9,12,24,36,60,84,120,180,240,360
<DATE>,,,,,,,,,,,,,,,,,,,,,
2016-04-01,56.20,134.91,8922.0,36.10,2310.2,215.5,11366.0,4.730,251.00,98.01,...,9.70,9.91,10.07,9.97,9.80,9.67,9.51,9.39,9.35,9.34
2016-05-01,54.45,136.45,9085.0,36.05,2380.0,213.2,11350.0,4.765,253.20,98.89,...,9.60,9.81,10.00,9.92,9.74,9.61,9.50,9.42,9.40,9.39
2016-06-01,54.05,135.94,8990.0,36.15,2378.0,212.5,11320.0,4.705,252.40,96.85,...,9.61,9.84,10.09,10.02,9.85,9.74,9.61,9.50,9.46,9.44
2016-11-01,52.06,131.38,8600.0,35.60,2257.9,208.9,10667.0,4.595,241.05,91.03,...,10.55,10.64,10.52,10.42,10.35,10.26,10.06,9.95,9.93,9.92
2016-12-01,53.25,133.54,8553.0,35.40,2223.5,210.1,10367.0,4.540,239.90,91.71,...,10.62,10.74,10.65,10.54,10.46,10.39,10.24,10.14,10.11,10.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,103.08,254.87,19630.0,56.80,6080.5,395.0,3371.5,3.288,451.40,248.67,...,5.44,5.49,5.68,5.84,6.11,6.31,6.50,6.66,6.74,6.82
2019-12-25,103.34,253.25,19890.0,57.30,6081.0,394.4,3416.0,3.500,454.00,248.04,...,5.39,5.43,5.65,5.81,6.10,6.31,6.49,6.66,6.74,6.82
2019-12-26,103.94,254.10,19220.0,57.90,6085.0,398.1,3449.0,3.446,452.50,248.24,...,5.40,5.46,5.67,5.80,6.05,6.27,6.48,6.66,6.75,6.83


In [320]:
log_risk_df = np.log(risk_df).diff()
log_risk_df

,AFLT,GAZP,GMKN,KMAZ,LKOH,PIKK,MGNT,RBCM,ROSN,SBER,...,9,12,24,36,60,84,120,180,240,360
<DATE>,,,,,,,,,,,,,,,,,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-05-01,-0.031634,0.011350,0.018105,-0.001386,0.029766,-0.010730,-0.001409,0.007372,0.008727,0.008939,...,-0.010363,-0.010142,-0.006976,-0.005028,-0.006141,-0.006224,-0.001052,0.003190,0.005333,0.005339
2016-06-01,-0.007373,-0.003745,-0.010512,0.002770,-0.000841,-0.003289,-0.002647,-0.012672,-0.003165,-0.020845,...,0.001041,0.003053,0.008960,0.010030,0.011230,0.013437,0.011512,0.008457,0.006363,0.005311
2016-11-01,-0.037513,-0.034120,-0.044351,-0.015331,-0.051825,-0.017086,-0.059416,-0.023657,-0.046011,-0.061974,...,0.093322,0.078165,0.041733,0.039144,0.049515,0.052012,0.045763,0.046281,0.048488,0.049597
2016-12-01,0.022601,0.016307,-0.005480,-0.005634,-0.015353,0.005728,-0.028527,-0.012042,-0.004782,0.007442,...,0.006613,0.009355,0.012282,0.011451,0.010572,0.012591,0.017734,0.018915,0.017965,0.017983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,-0.009654,-0.000510,-0.002341,-0.005268,-0.000247,0.001013,0.009387,-0.011491,0.000887,-0.000523,...,-0.007326,-0.005450,-0.003515,-0.003419,-0.004898,-0.006319,-0.006135,-0.005988,-0.004441,-0.001465
2019-12-25,0.002519,-0.006376,0.013158,0.008764,0.000082,-0.001520,0.013113,0.062483,0.005743,-0.002537,...,-0.009234,-0.010989,-0.005296,-0.005150,-0.001638,0.000000,-0.001540,0.000000,0.000000,0.000000
2019-12-26,0.005789,0.003351,-0.034266,0.010417,0.000658,0.009338,0.009614,-0.015549,-0.003309,0.000806,...,0.001854,0.005510,0.003534,-0.001723,-0.008230,-0.006359,-0.001542,0.000000,0.001483,0.001465


In [322]:
t = 1
vol = [(-1 + np.sqrt(1 + (1 / 1009) * np.sum((log_risk_df[log_risk_df.columns[i]] - t*np.mean(log_risk_df[log_risk_df.columns[i]]))**2))) / (t/2) for i in range(len(log_risk_df.columns))]
vol

[0.0003363680786643286,
 0.00022263484914519793,
 0.0002478627077189266,
 0.00023063651043608857,
 0.0002082510551275618,
 0.00015922427297976327,
 0.00033199229841684996,
 0.0006864559612309584,
 0.00021325533449401135,
 0.00031425224759162873,
 1.9707334205421034e-05,
 1.892297248096142e-05,
 1.648825343281146e-05,
 3.2760371751816564e-05,
 4.054345030013451e-05,
 0.0017398221114346413,
 0.0015505512608657845,
 0.0017711011586829528,
 8.302295768203294e-05,
 0.0001942056649917312,
 0.00037679647883415157,
 0.00032179392122255734,
 0.00027969926071991225,
 0.00024819053719138395,
 0.00018720182886244885,
 0.00016848211794906476,
 0.0001463856283714371,
 0.00012789560697124003,
 9.992366073019454e-05,
 7.474625599446227e-05,
 6.726635424714189e-05,
 7.480160537909342e-05]

In [11]:
def get_index():
    moex_ind = pd.read_csv('./index/IMOEX_160101_200101.csv')
    rts_ind = pd.read_csv('./index/RTSI_160101_200101.csv')
    zerobond = pd.read_csv('./zerobond.csv', sep=';')
    
    weeks_in_month = (365/7)*(1/12)
    maturity_rub = [3,6,9,12,24,36,60,84,120,180,240,360,]
    maturity_rub= [i * weeks_in_month for i in maturity_rub]
    
    interpolate_rub = CubicSpline(maturity_rub, ds_rate_rub)
    
    interval_rub = np.arange(0, 54, 2)
    
    df_rub_int = pd.DataFrame(data={'maturity_rub_2weeks': interval_rub, 'rub_act': interpolate_rub(interval_rub)})
    
    df_rub_usd_int=pd.concat([df_usd_int, df_rub_int], axis=1, sort=False)
    df_rub_usd_int=df_rub_usd_int.drop(['maturity_rub_2weeks'], axis=1)
    df_rub_usd_int=df_rub_usd_int.rename(columns={"maturity_usd_2weeks":"maturity"})
    df_rub_usd_int['maturity_frac'] = df_rub_usd_int['maturity']/54

    # s=0.0134
    s=1/0.01442

    new_rates = pd.concat(
        [
            df_rub_usd_int,
            df_rub_usd_int.diff(1).rename(columns={x:x.replace('act','diff') for x in df_rub_usd_int.columns})
        ],
        axis=1)

    new_rates.fillna(0, inplace=True)
    new_rates['fx_act']=(s*(1+new_rates['usd_act']*0.01)/(1+new_rates['rub_act']*0.01))
    new_rates['fx_diff'] =new_rates['fx_act'].diff()


    curve_rub_act = new_rates.loc[1:,'rub_act']
    curve_usd_act = new_rates.loc[1:,'usd_act']
    curve_fx_act = new_rates.loc[1:,'fx_act']

    curve_rub_diff = new_rates.loc[1:,'rub_diff']
    curve_usd_diff = new_rates.loc[1:,'usd_diff']
    curve_fx_diff = new_rates.loc[1:,'fx_diff']
    init = new_rates.loc[0,['rub_act','usd_act','fx_act']]


    return  (
        curve_rub_act,
        curve_usd_act,
        curve_fx_act,
        curve_rub_diff,
        curve_usd_diff,
        curve_fx_diff,
        init)
    
    
    
    
    def stoch_wrapper(decomp):
        def make_stoch(num):
            sigma=[0.03, 0.0093, 0.11]
            stoch_generator = np.dot(np.random.normal(size=(num,3)),decomp)*sigma
            return stoch_generator
        return make_stoch

    
stoch_generator = stoch_wrapper(get_decomp())

def simulate_hull_white(
    sim_number = 10,):
    rub_alpha=0.03
    sigma=[0.03, 0.0093, 0.11]
    k_fx=0.015
    dt=14/365
    timesteps = 26

    (
        curve_rub,
        curve_rub_df,
        init
        ) = get_rates()


    results = np.zeros(shape=(timesteps+1, 3, sim_number))

    passed_time=0

    for sim_ix in range(sim_number):
        results[0, :, sim_ix] = init
        stochs = stoch_generator(timesteps+1)
        for i, (rate_rub, df_rub, stoch_tuple) in enumerate(zip(curve_rub,curve_rub_df, stochs)):
            
            passed_time += dt

            theta_rub = df_rub + rub_alpha * rate_rub + (sigma[0]**2) * (1 - np.exp(-2 * rub_alpha * passed_time)) / 2 * rub_alpha

            results[i + 1, 0, sim_ix] = (theta_rub - rub_alpha* results[:, 0, sim_ix].sum()) * dt + stoch_tuple[0]

    return results

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,RTSI,D,04/01/16,00:00:00,755.16,761.68,741.67,749.28,204806463
1,RTSI,D,05/01/16,00:00:00,747.94,753.97,743.42,752.70,242929972
2,RTSI,D,06/01/16,00:00:00,753.06,754.04,735.62,736.82,210962101
3,RTSI,D,11/01/16,00:00:00,721.68,721.68,699.13,699.13,0
4,RTSI,D,12/01/16,00:00:00,689.33,706.31,681.46,695.32,0


# Оценка справедливой стоимости в зависимости от риск-факторов

# Оценка риска по портфелю

# Простая количественная валидация